In [ ]:
# pip install tensorflow==2.13.0
# pip install easyocr

<u>Gdrive</u>,
<br>whole_files_9000st.zip: //drive.google.com/file/d/1729iEumzufmOj3TIMj4yCvOtNR9AR9cG/view?usp=sharing
<br>only_workspace_9000st.zip: //drive.google.com/file/d/1GIe1Y9foZdvoPbLWhVvQ5sQ9y2obIEhP/view?usp=sharing
<br>my_model_9000st.zip: //drive.google.com/file/d/1i93ZIJPiQiUQ4984JUqnfUlc7dGz4kQv/view?usp=sharing

In [ ]:
# #  TF workspace zip with models
# gdown 1729iEumzufmOj3TIMj4yCvOtNR9AR9cG

In [ ]:
# # model is in .zip format. so unzip it
# import os
# import zipfile
# zip_ref = zipfile.ZipFile('/content/Whole_files_9000st.zip', 'r') # source
# zip_ref.extractall('/content') # Destination
# zip_ref.close()
# os.remove('/content/Whole_files_9000st.zip')

In [ ]:
# # delete unwanted files. need to make it before uploading zip
# os.remove('/content/detection_results.csv')
# os.remove('/content/Car_License_plate_detection_by_larxel.zip')
# import shutil
# shutil.rmtree('/content/Detection_Images')
# shutil.rmtree('/content/dataset')

<br>

# **Folder structures**

In [ ]:
import os

CUSTOM_MODEL_NAME = 'my_ssd_mobnet'
PRETRAINED_MODEL_NAME = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
TF_RECORD_SCRIPT_NAME = 'generate_tfrecord.py'
LABEL_MAP_NAME = 'label_map.pbtxt'

paths = {
    'WORKSPACE_PATH': os.path.join('Tensorflow', 'workspace'),
    'SCRIPTS_PATH': os.path.join('Tensorflow','scripts'),
    'APIMODEL_PATH': os.path.join('Tensorflow','models'),
    'ANNOTATION_PATH': os.path.join('Tensorflow', 'workspace','annotations'),
    'IMAGE_PATH': os.path.join('Tensorflow', 'workspace','images'),
    'MODEL_PATH': os.path.join('Tensorflow', 'workspace','models'),
    'PRETRAINED_MODEL_PATH': os.path.join('Tensorflow', 'workspace','pre-trained-models'),
    'CHECKPOINT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME),
    'OUTPUT_PATH': os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'export'),
    'TFJS_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfjsexport'),
    'TFLITE_PATH':os.path.join('Tensorflow', 'workspace','models',CUSTOM_MODEL_NAME, 'tfliteexport'),
    'PROTOC_PATH':os.path.join('Tensorflow','protoc')
 }

files = {
    'PIPELINE_CONFIG':os.path.join('Tensorflow', 'workspace','models', CUSTOM_MODEL_NAME, 'pipeline.config'),
    'TF_RECORD_SCRIPT': os.path.join(paths['SCRIPTS_PATH'], TF_RECORD_SCRIPT_NAME),
    'LABELMAP': os.path.join(paths['ANNOTATION_PATH'], LABEL_MAP_NAME)
}

In [ ]:
# if os.name=='nt':
#     !pip install wget
#     import wget

In [ ]:
# # Install Tensorflow Object Detection
# if os.name=='posix':
#      apt-get install protobuf-compiler
#     cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && cp object_detection/packages/tf2/setup.py . && python -m pip install .
#     pip install tensorflow==2.13.0

# if os.name=='nt':
#     url="https://github.com/protocolbuffers/protobuf/releases/download/v3.15.6/protoc-3.15.6-win64.zip"
#     wget.download(url)
#     move protoc-3.15.6-win64.zip {paths['PROTOC_PATH']}
#     cd {paths['PROTOC_PATH']} && tar -xf protoc-3.15.6-win64.zip
#     os.environ['PATH'] += os.pathsep + os.path.abspath(os.path.join(paths['PROTOC_PATH'], 'bin'))
#     cd Tensorflow/models/research && protoc object_detection/protos/*.proto --python_out=. && copy object_detection\\packages\\tf2\\setup.py setup.py && python setup.py build && python setup.py install
#     cd Tensorflow/models/research/slim && pip install -e .

In [ ]:
# VERIFICATION_SCRIPT = os.path.join(paths['APIMODEL_PATH'], 'research', 'object_detection', 'builders', 'model_builder_tf2_test.py')
# # Verify Installation
# python {VERIFICATION_SCRIPT}

<br>

# Load Train Model From Checkpoint

In [ ]:
import os
import tensorflow as tf
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder
from object_detection.utils import config_util

In [ ]:
# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(files['PIPELINE_CONFIG'])
detection_model = model_builder.build(model_config=configs['model'], is_training=False)

# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(paths['CHECKPOINT_PATH'], 'ckpt-9')).expect_partial()  # change checkpoint

@tf.function
def detect_fn(image):
    image, shapes = detection_model.preprocess(image)
    prediction_dict = detection_model.predict(image, shapes)
    detections = detection_model.postprocess(prediction_dict, shapes)
    return detections

<br>

# Detect from an Image

In [ ]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
category_index = label_map_util.create_category_index_from_labelmap(files['LABELMAP'])

In [ ]:
def find_plate(img_path, min_score_thresh=.6):
    # IMAGE_PATH = os.path.join(paths['IMAGE_PATH'], 'test', 'Cars420.png') # Replace file name
    # IMAGE_PATH = img_path

    # img = cv2.imread(IMAGE_PATH)
      # img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY) # added
    image_np = np.array(img_path)     #img

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detect_fn(input_tensor)

    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy() for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes']+label_id_offset,
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=5,
        min_score_thresh=min_score_thresh, # min 0.8
        agnostic_mode=False
    )

    plt.imshow(cv2.cvtColor(image_np_with_detections, cv2.COLOR_BGR2RGB))
    plt.show()
    return image_np_with_detections, detections

In [ ]:
image_np_with_detections, detections = find_plate(os.path.join(paths['IMAGE_PATH'], 'test', 'Cars423.png'), 0.3 )

# Apply OCR to Detection

In [ ]:
import easyocr

In [ ]:
detection_threshold = 0.7
region_threshold = 0.5 # change to o.6

In [ ]:
def filter_text(region, ocr_result, region_threshold=0.6):
    rectangle_size = region.shape[0]*region.shape[1]

    plate = []
    for result in ocr_result:
        length = np.sum(np.subtract(result[0][1], result[0][0]))
        height = np.sum(np.subtract(result[0][2], result[0][1]))

        if length*height / rectangle_size > region_threshold:
            plate.append(result[1])
    return plate

def ocr_it(image, detections, detection_threshold=0.7, region_threshold=0.6):

    # Scores, boxes and classes above threhold
    scores = list(filter(lambda x: x> detection_threshold, detections['detection_scores']))
    boxes = detections['detection_boxes'][:len(scores)]
    classes = detections['detection_classes'][:len(scores)]

    # Full image dimensions
    width = image.shape[1]
    height = image.shape[0]

    # Apply ROI filtering and OCR
    for idx, box in enumerate(boxes):
        roi = box*[height, width, height, width]
        region = image[int(roi[0]):int(roi[2]),int(roi[1]):int(roi[3])]
        reader = easyocr.Reader(['en'])
        ocr_result = reader.readtext(region)

        text = filter_text(region, ocr_result, region_threshold)

        plt.imshow(cv2.cvtColor(region, cv2.COLOR_BGR2RGB))
        plt.show()
        print(text)
        return text, region

In [ ]:
text, region = ocr_it(image_np_with_detections, detections, detection_threshold, region_threshold)

# Save Results

In [ ]:
import csv
import uuid

In [ ]:
os.mkdir(os.path.join('/content', 'Detection_Images'))

In [ ]:
def save_results(text, region, csv_filename, folder_path):
    img_name = '{}.jpg'.format(uuid.uuid1())

    cv2.imwrite(os.path.join(folder_path, img_name), region)

    with open(csv_filename, mode='a', newline='') as f:
        csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow([img_name, text])

In [ ]:
region

In [ ]:
save_results(text, region, 'detection_results.csv', 'Detection_Images')

<br>

# From Video

In [ ]:
import cv2
from matplotlib import pyplot as plt
import IPython.display as ipd

<br>

In [ ]:
video_path = '/content/traffic_15s_oneway.mp4'

In [ ]:
ipd.Video(video_path)

In [ ]:
from IPython.display import HTML

HTML("""
    <video alt="test" controls>
        <source src="video_path" type="video/mp4">
    </video>
""")

In [ ]:
cap = cv2.VideoCapture(video_path)
print(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print(cap.get(cv2.CAP_PROP_FPS))
print(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
print(cap.get(cv2.CAP_PROP_FRAME_WIDTH))


In [ ]:
i = 0
# os.mkdir('out')
while(cap.isOpened()):
  ret, frame = cap.read()
  if ret:
    cv2.imwrite(f'out/{i}.png', frame)
  print(i)
  i +=1

In [ ]:
cap = cv2.VideoCapture(video_path)
# ret, frame = cap.read()

# # img = cv2.imread(frame)
# # img.shape
# # type(img)
# # type(frame)
# a, v = find_plate(frame, 0.1)

ret, frame2 = cap.read()
# a,c = find_plate(frame2, 0.1)

# for i in range(5):
while cap.isOpened():
  ret, frame = cap.read()
  if not ret:
    break
  if cap.get(cv2.CAP_PROP_POS_FRAMES) % 100 == 0:
    a, b = find_plate(frame, 0.01)
    # print('frme')
    m = ocr_it(a, b)
    # print(f' , {c} .')
    # cv2.imshow(m)

# cap.get(cv2.CAP_PROP_POS_FRAMES)

In [ ]:
# ret, frame = cap.read()
# find_plate(os.path.join('content', 'out', 'o.png'))

original_img = plt.imread('/content/Tensorflow/workspace/images/test/Cars415.png')
vid_img = plt.imread('/content/out/100.png')


# plt.imshow(original_img)
# plt.imshow(vid_img)

print('original', original_img.shape)
print(vid_img.shape)

In [ ]:
a, b = find_plate('/content/Tensorflow/workspace/images/test/Cars416.png')
print('n')

In [ ]:
a, b = find_plate('/content/out/405.png', 0.1)

<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>

In [ ]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/4513830-sd_640_360_30fps.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
# แปลงไฟล์เป็น string ในรูปแบบ data url
from IPython.display import HTML
from base64 import b64encode
mp4 = open('/content/｜ Indian beautiful Vehicles are running on road  ｜ Indian cars ｜ Indian trucks ｜ Indian road ｜ [UgkxcGO07DNJoN1ruVaBDIxXwh0ROpPHo7O7].mkv','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

In [ ]:
# แสดงผลด้วย vdo tag    ใครงง ก็ไปเรียน HTML ใน Sololearn ได้
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [ ]:
import cv2
import os

def preprocess_frame(frame):
    # Your preprocessing steps here
    # For example, you can convert the frame to grayscale
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Or apply a Gaussian blur
    blurred_frame = cv2.GaussianBlur(gray_frame, (5, 5), 0)
    return blurred_frame

def process_video(input_video_path, output_video_path):
    # Open the input video file
    input_video = cv2.VideoCapture(input_video_path)

    # Get the frame rate of the input video
    frame_rate = int(input_video.get(cv2.CAP_PROP_FPS))

    # Create a VideoWriter object to save the output video
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    output_video = cv2.VideoWriter(output_video_path, fourcc, frame_rate, (int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    # Read each frame from the input video
    while input_video.isOpened():
        ret, frame = input_video.read()

        # Check if the frame was read correctly
        if not ret:
            break

        # Preprocess the frame
        processed_frame = preprocess_frame(frame)

        # Write the processed frame to the output video
        output_video.write(processed_frame)

    # Release the input and output video files
    input_video.release()
    output_video.release()

# Example usage
input_video_path = 'input_video.mp4'
output_video_path = 'output_video.mp4'
process_video(input_video_path, output_video_path)